# Comparing LLM Path Extractors for Knowledge Graph Construction

In this notebook, we'll compare three different LLM Path Extractors from llama_index:
1. SimpleLLMPathExtractor
2. SchemaLLMPathExtractor
3. DynamicLLMPathExtractor (New)

We'll use a Wikipedia page as our test data and visualize the resulting knowledge graphs using Pyvis.

## Setup and Imports

In [ ]:
!pip install llama_index pyvis wikipedia

In [1]:
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

import wikipedia

import os

c:\Users\maebo\miniconda3\envs\knowledge_graph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nest_asyncio
nest_asyncio.apply()

## Set up LLM Backend

In [3]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY_HERE"

# Set up global configurations
llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo", reuse_client=False)

Settings.llm = llm
Settings.chunk_size = 2048
Settings.chunk_overlap = 20

## Fetch Wikipedia Content

In [ ]:
def get_wikipedia_content(title):
    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation page. Options: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"Page '{title}' does not exist.")
    return None

In [4]:
wiki_title = "Barack Obama"
content = get_wikipedia_content(wiki_title)

if content:
    document = Document(text=content, metadata={"title": wiki_title})
    print(f"Fetched content for '{wiki_title}' (length: {len(content)} characters)")
else:
    print("Failed to fetch Wikipedia content.")

Fetched content for 'Barack Obama' (length: 83713 characters)


## 1. SimpleLLMPathExtractor

In [5]:
kg_extractor = SimpleLLMPathExtractor(
    llm=llm,
    max_paths_per_chunk=20,
    num_workers=4
)

simple_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes = False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

simple_index.property_graph_store.save_networkx_graph(name="./SimpleGraph.html")
simple_index.property_graph_store.get_triplets(entity_names=['Barack Obama','Obama'])

Extracting paths from text: 100%|██████████| 11/11 [00:11<00:00,  1.03s/it]


[(EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '9ff27c00-1418-4349-a443-227c1521dffc'}, name='Obama'),
  Relation(label='Received', source_id='Obama', target_id='Nobel peace prize in 2009', properties={'title': 'Barack Obama', 'triplet_source_id': '9ff27c00-1418-4349-a443-227c1521dffc'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '9ff27c00-1418-4349-a443-227c1521dffc'}, name='Nobel peace prize in 2009')),
 (EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '9ff27c00-1418-4349-a443-227c1521dffc'}, name='Obama'),
  Relation(label='Received', source_id='Obama', target_id='Ambassador of humanity award in 2014', properties={'title': 'Barack Obama', 'triplet_source_id': '9ff27c00-1418-4349-a443-227c1521dffc'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '9ff27c00-1418

## 3. DynamicLLMPathExtractor

### Without intial ontology :
Here, we let the LLM define the ontology on the fly, giving it full freedom to label the nodes as it best sees fit.

In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=None, # Let the LLM infer entities and their labels (types) on the fly
    allowed_relation_types=None # Let the LLM infer relationships on the fly
)

dynamic_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes = False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index.property_graph_store.save_networkx_graph(name="./DynamicGraph.html")

In [7]:
dynamic_index.property_graph_store.get_triplets(entity_names=['Barack Obama','Obama'])

[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'fb19b8cb-cb66-45e3-8e97-73a725758810'}, name='Obama'),
  Relation(label='TAUGHT_AT', source_id='Obama', target_id='University of Chicago Law School', properties={'title': 'Barack Obama', 'triplet_source_id': 'c783aefa-2ff9-4b64-a77c-2ad42146502b'}),
  EntityNode(label='SCHOOL', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'c783aefa-2ff9-4b64-a77c-2ad42146502b'}, name='University of Chicago Law School')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'fb19b8cb-cb66-45e3-8e97-73a725758810'}, name='Obama'),
  Relation(label='RESIGNED_FROM', source_id='Obama', target_id='Illinois Senate', properties={'title': 'Barack Obama', 'triplet_source_id': '5e0ff7d7-5e18-44f6-bcd2-ba5192ccfbe1'}),
  EntityNode(label='POSITION', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '5e0ff7d7-5e1

### With initial ontology for guided KG extraction : 
Here, we have partial knowledge of what we want to detect, we know the article is about Barack Obama, so we define some entities and relations that could help guide the LLM in the labeling process as it detects the entities and relations. This doesn't guarantee that the LLM will use them, it simply guides it and gives it some ideas. It will still be up to the LLM to decide whether it uses the entities and relations we provide or not. 

In [10]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=["POLITICIAN", "POLITICAL_PARTY", "LEGISLATION"],
    allowed_relation_types=["PRESIDENT_OF", "RAN_AGAINST"] 
)

dynamic_index_2 = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes = False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index_2.property_graph_store.save_networkx_graph(name="./DynamicGraph_2.html")

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 71.15it/s]
Extracting and inferring knowledge graph from text:   0%|          | 0/11 [00:00<?, ?it/s]Retrying llama_index.llms.openai.base.OpenAI._achat in 0.9275188322796465 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.5669553061972603 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.12775166406343308 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.706059731182484 seconds as it raised APIConnectionError: Connection error..
Extracting and inferring knowledge graph from text:   9%|▉         | 1/11 [01:41<16:51, 101.12s/it]

Error during extraction: Connection error.
Error during extraction: Connection error.


Extracting and inferring knowledge graph from text:  27%|██▋       | 3/11 [01:41<03:02, 22.82s/it] 

Error during extraction: Connection error.


Extracting and inferring knowledge graph from text:  36%|███▋      | 4/11 [01:42<01:40, 14.35s/it]

...

Error during extraction: Connection error.


Extracting and inferring knowledge graph from text:  82%|████████▏ | 9/11 [03:43<00:37, 18.66s/it]

Error during extraction: Connection error.


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.7909673069771449 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.09888813077167624 seconds as it raised APIConnectionError: Connection error..
Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [04:57<00:00, 27.08s/it]

Error during extraction: Connection error.


In [11]:
dynamic_index_2.property_graph_store.get_triplets(entity_names=['Barack Obama','Obama'])

[(EntityNode(label='POLITICIAN', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '99c4bd8d-9d5d-45a3-b89d-e171c48bed99'}, name='Barack Obama'),
  Relation(label='ENROLLED_IN', source_id='Barack Obama', target_id='Harvard Law School', properties={'title': 'Barack Obama', 'triplet_source_id': '809d242e-03f7-4307-8b04-d0a1770ca29f'}),
  EntityNode(label='UNIVERSITY', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '809d242e-03f7-4307-8b04-d0a1770ca29f'}, name='Harvard Law School')),
 (EntityNode(label='POLITICIAN', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '99c4bd8d-9d5d-45a3-b89d-e171c48bed99'}, name='Barack Obama'),
  Relation(label='RECEIVED', source_id='Barack Obama', target_id='Ambassador of Humanity Award', properties={'title': 'Barack Obama', 'triplet_source_id': '99c4bd8d-9d5d-45a3-b89d-e171c48bed99'}),
  EntityNode(label='AWARD', embedding=None, properties={'title': 'Barack Obama', 'triplet_source

# 3 - SchemaLLMPathExtractor

In [8]:
kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    strict=False, # Set to False to showcase why it's not going to be the same as DynamicLLMPathExtractor
    possible_entities=None, # USE DEFAULT ENTITIES (PERSON, ORGANIZATION... etc)
    possible_relations=None, # USE DEFAULT RELATIONSHIPS
    num_workers=4
)

schema_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes = False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

schema_index.property_graph_store.save_networkx_graph(name="./SchemaGraph.html")


Extracting paths from text with schema: 100%|██████████| 11/11 [00:41<00:00,  3.79s/it]


In [9]:
schema_index.property_graph_store.get_triplets(entity_names=['Barack Obama','Obama'])

[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '5e9da23f-b3d6-4aed-a5e7-065a1c6baf88'}, name='Barack Obama'),
  Relation(label='WORKED_ON', source_id='Barack Obama', target_id='Middle Class Tax Relief and Job Creation Act of 2012', properties={'title': 'Barack Obama', 'triplet_source_id': '5e9da23f-b3d6-4aed-a5e7-065a1c6baf88'}),
  EntityNode(label='CONCEPT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '5e9da23f-b3d6-4aed-a5e7-065a1c6baf88'}, name='Middle Class Tax Relief and Job Creation Act of 2012')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '5e9da23f-b3d6-4aed-a5e7-065a1c6baf88'}, name='Barack Obama'),
  Relation(label='WORKED_ON', source_id='Barack Obama', target_id='Illinois Senate (1997–2004)', properties={'title': 'Barack Obama', 'triplet_source_id': '402e476c-59b3-45ff-b4e0-f6e7aea9127a'}),
  EntityNode(label='EVENT', embedding=Non

## Comparison and Analysis

Let's compare the results of the three extractors:

1. **SimpleLLMPathExtractor**: This extractor creates a basic knowledge graph without any predefined schema. It may produce a larger number of diverse relationships but might lack consistency in entity and relation naming.


3. **DynamicLLMPathExtractor**: 
    - This new extractor combines the flexibility of SimpleLLMPathExtractor with some initial guidance from a schema. It can expand beyond the initial entity and relation types, potentially producing a rich and diverse graph while maintaining some level of consistency. 
    - Not giving it any entities or relations to start with in the input gives the LLM complete freedom to infer the schema on the fly as it best sees fit. This is going to vary based on the LLM and the temperature used.

3. **SchemaLLMPathExtractor**: With a predefined schema, this extractor produces a more structured graph. The entities and relations are limited to those specified in the schema, which can lead to a more consistent but potentially less comprehensive graph. Even if we set "strict" to false, the extracted KG Graph doesn't reflect the LLM's pursuit of trying to find new entities and types that fall outside of the input schema's scope.


Key observations:
- The SimpleLLMPathExtractor graph might have the most diverse set of entities and relations.
- The SchemaLLMPathExtractor graph should be the most consistent but might miss a lot of relationships that don't fit the predefined schema, even if we don't impose a strict validation of the schema.
- The DynamicLLMPathExtractor graph should show a balance between diversity and consistency, potentially capturing important relationships that the schema-based approach might miss while still maintaining some structure.

The choice between these extractors depends on the specific use case:
- Use SimpleLLMPathExtractor for exploratory analysis where you want to capture a wide range of potential relationships for RAG applications, without caring about the entity types.
- Use SchemaLLMPathExtractor when you have a well-defined domain and want to ensure consistency in the extracted knowledge.
- Use DynamicLLMPathExtractor when you want a balance between structure and flexibility, allowing the model to discover new entity and relation types while still providing some initial guidance. This one is especially useful if you want a KG with labeled (typed) entities but don't have an input Schema (or you've partially defined the schema as a starting base).